# SHELFIE

Run all neccessary pip installs

In [ ]:
%pip uninstall -qqy jupyterlab 
# Remove unused conflicting packages
%pip install -U -q "google-genai==1.7.0"
%pip install faker
%pip install google-api-python-client
%pip install dotenv
%pip install -qU "langgraph==0.3.21" "langchain-google-genai==2.1.2" "langgraph-prebuilt==0.1.7"

Do all the neccessary imports to declutter code below. Also verify genai is being imported correctly.

In [ ]:
import sqlite3
import random
from faker import Faker
from google import genai
from google.genai import types
from google.api_core import retry
from typing import Final
from IPython.display import Markdown, display, Image
from pprint import pprint
print(genai.__version__)
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages.ai import AIMessage
from typing import Literal
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from collections.abc import Iterable
from langchain_core.messages.tool import ToolMessage

Set up `GOOGLE_API_KEY` for both Kaggle and local development.

In [ ]:
import os
from dotenv import load_dotenv

try:
    from kaggle_secrets import UserSecretsClient
    IS_KAGGLE = True
except ImportError:
    IS_KAGGLE = False

# Load environment variables from .env file for local setup
if not IS_KAGGLE:
    load_dotenv()

# Fetch the Google API key
if IS_KAGGLE:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
else:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set it in the appropriate environment.")

In [ ]:
NUM_BOOKS: Final[int] = 30

In [ ]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
# Remove existing database file (if it exists)
db_file = "library.db"
if os.path.exists(db_file):
    os.remove(db_file)

In [ ]:
from dataclasses import dataclass

@dataclass
class Book:
    name: str
    genre: str
    description: str
    author: str

In [ ]:
def generate_books_list(num_books: int) -> list[Book]:
    """
    Asks Gemini to create a list of
    
    return: A list of Book objects.
    """
    prompt = """
    Generate a list of exactly {num_books} books of any genre. For each book, provide the name, genre, a description of about 1000 characters, and the author.
    Do not include the author's name in the description. Return the result as a JSON array of book objects with the keys: "name", "genre", "description", and "author".
    Examples of a "name", "genre", "description", and "author" are provided below:
    1) "To Kill a Mockingbird", "Fiction", "A gripping tale set in the racially charged American South, exploring themes of justice, morality, and the loss of innocence. Perfect for readers who appreciate profound social commentary interwoven with compelling storytelling.", "Harper Lee"
    2) "1984", "Dystopian", "A chilling vision of a totalitarian future where surveillance and control pervade every aspect of life. Ideal for fans of thought-provoking and cautionary tales.", "George Orwell"
    """

    if num_books < 1:
        raise Exception("Need to generate at least one Book.")
    
    prompt = prompt.format(num_books = num_books)
    client = genai.Client(api_key=GOOGLE_API_KEY)
    
    structured_output_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema={
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "genre": {"type": "string"},
                    "description": {"type": "string"},
                    "author": {"type": "string"}
                },
                "required": ["name", "genre", "description", "author"]
            }
        },
        temperature=0.3
    )

    response_len = 0

    while response_len != num_books:
        chat = client.chats.create(model='gemini-2.0-flash')
        response = chat.send_message(
          message=prompt,
          config=structured_output_config,
        )
        book_objects = [Book(**book) for book in response.parsed]
        response_len = len(book_objects)
    
    return book_objects

Do a basic check to see that an exception is raised when the user provides `num_books` < 1.

In [ ]:
try:
    generate_books_list(0)
except:
    print("Expected error when calling generate_books_list with 0 books.\n\n")

Call `generate_books_list` with `NUM_BOOKS`. The time it takes to run the line of code is non-deterministic. This is the number of books produced by gemini is not consitent.

In [ ]:
books_list = generate_books_list(NUM_BOOKS)

Do some validation on the generated output. Verify if the generated list is indeed of length 30. If it's not try running it again. Print the top 5 for inspection.

In [ ]:
assert(len(books_list) == NUM_BOOKS)
assert(all([True for book in books_list if type(book) == Book]))
pprint(books_list[:5])

In [ ]:
# Initialize Faker instance
fake = Faker()

# Create SQLite database and tables
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    password_hash TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Book (
    book_id INTEGER PRIMARY KEY,
    name TEXT,
    genre TEXT,
    description TEXT,
    author TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Stock (
    stock_id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER,
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS BorrowingHistory (
    entry_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    stock_id INTEGER,
    borrowed_on DATE,
    returned_on DATE,
    due_by DATE,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (stock_id) REFERENCES Stock(stock_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Reservations (
    reservation_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    book_id INTEGER,
    status TEXT,
    last_updated DATE,
    reserved_on DATE,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Waitlist (
    book_id INTEGER,
    waitlisted_on DATE,
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
)
""")

passwords = ["ISNA2018", "ISNA2019", "ISNA2020", "ISNA2021", "ISNA2022"]
# Generate and insert sample data for Users table
for user_id in range(1, 6):
    cursor.execute("""
    INSERT INTO Users (user_id, name, email, password_hash)
    VALUES (?, ?, ?, ?)
    """, (user_id, fake.name(), fake.email(), passwords[user_id - 1]))

# Insert real books into Book table
for book_id, book in enumerate(books_list, start=1):
    cursor.execute("""
    INSERT INTO Book (book_id, name, genre, description, author)
    VALUES (?, ?, ?, ?, ?)
    """, (book_id, book.name, book.genre, book.description, book.author))

# Generate and insert sample data for Stock table
for book_id in range(1, 31):
    num_books = random.randint(3, 5)
    j = 0
    while (j < num_books):
        cursor.execute("""
        INSERT INTO Stock (book_id)
        VALUES (?)
        """, (book_id, ))
        j += 1

# Generate and insert sample data for BorrowingHistory table
for entry_id in range(1, 21):
    cursor.execute("""
    INSERT INTO BorrowingHistory (entry_id, user_id, stock_id, borrowed_on, returned_on, due_by)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (entry_id, random.randint(1, 5), random.randint(1, 50), fake.date_between(start_date="-1y", end_date="-1d"), fake.date_between(start_date="+1d", end_date="+1y"), fake.date_between(start_date="+1d", end_date="+1y")))

# Generate and insert sample data for Reservations table
for reservation_id in range(1, 21):
    cursor.execute("""
    INSERT INTO Reservations (reservation_id, user_id, book_id, status, last_updated, reserved_on)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (reservation_id, random.randint(1, 5), random.randint(1, 50), random.choice(["Active", "Fulfilled", "Expired"]), fake.date_between(start_date="-1y", end_date="today"), fake.date_between(start_date="-1y", end_date="today")))

# Generate and insert sample data for Waitlist table
for book_id in range(1, 6):
    cursor.execute("""
    INSERT INTO Waitlist (book_id, waitlisted_on)
    VALUES (?, ?)
    """, (book_id, fake.date_between(start_date="-1y", end_date="today")))

# Commit changes and close connection
conn.commit()
conn.close()

print("Sample data has been inserted into the SQLite database.")


In [ ]:
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

# Query all tables to display the data
cursor.execute("SELECT * FROM Users")
print("Users:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Book")
print("\nBooks:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Stock")
print("\nStock:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM BorrowingHistory")
print("\nBorrowingHistory:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Reservations")
print("\nReservations:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Waitlist")
print("\nWaitlist:")
for row in cursor.fetchall():
    print(row)

In [ ]:
class BookState(TypedDict):
    """State representing the customer's conversation about borrowing library books."""

    # The chat conversation. This preserves the conversation history
    # between nodes. The `add_messages` annotation indicates to LangGraph
    # that state is updated by appending returned messages, not replacing
    # them.
    messages: Annotated[list, add_messages]

    user_id: int

    # The customer's in-progress order.
    reservedBooks: list[int]

    # List of book ids that the customer is on the waitlist for
    waitlist: list[int]

    # Flag indicating that the conversation has completed.
    finished: bool


SYSTEM_INSTRUCTION = (
    "You are Shelfie, a friendly and knowledgeable library assistant chatbot.\n"
    "You help users explore, reserve, and waitlist books available in our library through this chat interface.\n"
    "You have detailed knowledge of all books in our collection, including plots, characters, themes, and the ability to recall specific passages with page and book references.\n\n"
    "Your capabilities include:"
    "\n\n"
    "- Searching for books by title, author, genre, or user-provided descriptions or themes.\n"
    "- Informing users if a book is available, and offering to reserve it for 24 hours for physical pickup.\n"
    "- If a book is unavailable, offering to add the user to a waitlist — only after confirming their intent.\n"
    "- Recommending highly-rated or popular books, based on borrowing history and inventory availability.\n"
    "- Suggesting alternative books if the requested one is unavailable.\n"
    "- You can also provide a rating out of 5. This rating will be based on multiple factors:\n"
    "  - Frequency of that book's borrowing history\n"
    "  - Reviews on the internet. These can be accumulated from multiple sources such as Google, Amazon, GoodReads, etc.\n\n"
    "Your responses must always be helpful, concise, and strictly related to library topics.\n"
    "If a user asks something outside the scope of the library, politely decline and redirect them.\n"
    "Always confirm user intent before adding books to reservations or waitlists.\n"
    "Ask clarifying questions if a user’s request is ambiguous.\n"
    "If a requested feature is not implemented yet, respond with: 'This feature isn't available yet, but we're working on it!'"
)

WELCOME_MSG = (
    "Welcome to SHELFIE. I can help you find and recommend books with the ability to explore, reserve or waitlist books with ease."
    "You can search by title, description, author, or an excerpt. Let's find your next great read!"
    "You must login to reserve or waitlist books. Please provide your user ID and password to login."
    "You can type `q` to quit chatting with me anytime but that would make me sad :("
)

In [ ]:
# Create SQLite database and tables
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

def list_tables() -> list[str]:
    """
    Retrieve the names of all tables in the database.
    :return: A list of table names.
    """
    
    print(' - DB CALL: list_tables()')

    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

In [ ]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """
    Look up the table schema.
    param table_name: The name of the table to describe.
    :return: A list of columns, where each entry is a tuple of (column, type).
    """

    print(f' - DB CALL: describe_table({table_name})')

    cursor = conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    return [(col[1], col[2]) for col in schema]


describe_table("Users")

In [ ]:
def execute_query(sql: str) -> list[list[str]]:
    """
    Execute an SQL statement, returning the results.
    param sql: The SQL statement to execute.
    :return: The results of the query.
    """
    print(f' - DB CALL: execute_query({sql})')

    cursor = conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("SELECT * FROM Users")

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

def login_with_welcome_msg(state: BookState) -> BookState:
    """
    The Login node itself. Present the user with a welcome message, ask for their user ID and password,
    and verify the credentials against the database.
    :param state: The current state of the conversation.
    :return: The updated state with the welcome message and user ID and password prompts.
    """
    if not state["messages"]:
        # Start with the welcome message.
        new_output = AIMessage(content=WELCOME_MSG)
        return state | {"messages": [new_output]}

    # Continue the conversation with the Gemini model.
    user_input = state["messages"][-1].content.strip()

    if "username" not in state:
        # Ask for the username if not already provided.
        new_output = AIMessage(content="Please enter your username:")
        return state | {"messages": state["messages"] + [new_output], "username": user_input}

    if "password" not in state:
        # Ask for the password if not already provided.
        new_output = AIMessage(content="Please enter your password:")
        return state | {"messages": state["messages"] + [new_output], "password": user_input}

    # Verify the username and password against the database.
    cursor.execute("SELECT password_hash FROM Users WHERE name = ?", (state["username"],))
    result = cursor.fetchone()

    if result and result[0] == state["password"]:
        new_output = AIMessage(content="Login successful! How can I assist you today?")
        return state | {"messages": state["messages"] + [new_output], "finished": False}
    else:
        new_output = AIMessage(content="Invalid username or password. Please try again.")
        return state | {"messages": state["messages"] + [new_output], "username": None, "password": None}


# Update the graph to include the modified login node.
graph_builder = StateGraph(BookState)
graph_builder.add_node("login", login_with_welcome_msg)
graph_builder.add_edge(START, "login")

In [ ]:
def maybe_route_to_chatbot(state: BookState) -> str:
    """
    Route the conversation to the chatbot node if the user is logged in.
    :param state: The current state of the conversation.
    :return: The next node to route to.
    """
    if state.get("user_id"):
        # User is logged in, route to the chatbot node.
        return "chatbot"
    else:
        return "login"

In [ ]:
def chatbot_node(state: BookState) -> BookState:
    """
    The Chatbot node where the user interacts with the library assistant after successful login.
    :param state: The current state of the conversation.
    :return: The updated state.
    """
    new_output = AIMessage(content="You are now in the chatbot interface. How can I assist you?")
    return state | {"messages": state["messages"] + [new_output]}


# Add the chatbot node to the graph
graph_builder.add_node("chatbot", chatbot_node)

# Add conditional edges from the login node
graph_builder.add_conditional_edges("login", maybe_route_to_chatbot)

# Compile the updated graph
chat_graph = graph_builder.compile()

# Visualize the updated graph
Image(chat_graph.get_graph().draw_mermaid_png())